In [44]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import date
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

train = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/train.csv")
test = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/test.csv")
submit = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/submission.csv")
weather = pd.read_csv("/kaggle/input/weather-data/training_data_with_weather_info_week_4.csv")

/kaggle/input/weather-data/training_data_with_weather_info_week_1.csv
/kaggle/input/weather-data/__resultx__.html
/kaggle/input/weather-data/custom.css
/kaggle/input/weather-data/__notebook__.ipynb
/kaggle/input/weather-data/__results__.html
/kaggle/input/weather-data/__output__.json
/kaggle/input/weather-data/training_data_with_weather_info_week_4.csv
/kaggle/input/weather-data/__results___files/__results___3_0.png
/kaggle/input/weather-data/__results___files/__results___2_1.png
/kaggle/input/covid19-global-forecasting-week-4/train.csv
/kaggle/input/covid19-global-forecasting-week-4/submission.csv
/kaggle/input/covid19-global-forecasting-week-4/test.csv
/kaggle/input/covid19formattedweatherjan22march24/covid_dataset.csv


# Data Transformation
1.  Combine the 'Country_Region' and 'Province_State' columns into 'country_province'.
2.  Calculate the cumulative cases and fatalities for each country_province as 'cumCases' and 'cumDeath'
3.  Chagnge the datatype of 'Date' to datetime  
~~4.  Remove the zero days for each country_province.~~ **Keep zeros to see if it fits the model better**
5.  Add weather data for each day
6.  Add 'prevCases' and 'prevDeath' for train data
7.  Add 'Days' since Jan-22-2020


In [45]:
def dataTrans(df, purpose):
    # 1. Combine the Country_Region and Province_State columns into country_province.
    df.Province_State[df['Province_State'].isnull()] = '' # change the null to empty string
    df['country_province'] = df.apply(lambda x: x.Country_Region if x.Province_State == '' else x.Country_Region+'_'+x.Province_State, axis = 1)
    # 3. Chagnge the datatype of Date to datetime
    df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)
    # 4. Remove the zero case days for each country_province.
    #df_filtered = df_cum[df_cum.ConfirmedCases > 0.0]
    #df_filtered.head(10)  
    # 6.  Add a column for DaysSince jan-22-2020
    df["Days"] = (df.Date - pd.Timestamp('2020-01-22 00:00:00')).dt.days
    
    
    ##################################################################################################
    # for training dataset only
    if (purpose == 'train'):
        # 2. Calculate the cumulative cases and fatalities for each country_province for the train data
        # 5.  Add column for prevCases and prevDeath
        cumCases = pd.Series()
        cumDeath = pd.Series()
        prevCases = pd.Series()
        prevDeath = pd.Series()
        for region in df.country_province.unique():
            cases = df.ConfirmedCases[df.country_province==region].cumsum()
            death = df.Fatalities[df.country_province==region].cumsum()
            cumCases = pd.concat([cumCases,cases])
            cumDeath = pd.concat([cumDeath,death])
            prevCases = pd.concat([prevCases,pd.Series([0]),cases.iloc[:len(cases)-1]])
            prevDeath = pd.concat([prevDeath,pd.Series([0]),death.iloc[:len(death)-1]])
        prevCases = prevCases.reset_index(drop=True)
        prevDeath = prevDeath.reset_index(drop=True)
        #print(len(cumCases), len(cumDeath), df.shape[0])
        df_cum = pd.concat([df,cumCases,cumDeath,prevCases,prevDeath], axis=1)
        #print(df.shape, df_cum.shape)
        df_cum = df_cum.rename(columns={0:'cumCases', 1:'cumDeath', 2:'prevCases', 3:'prevDeath'})
        return df_cum#, df_filtered
    ######################################################################################################
    else:
        return df

In [ ]:
df_train = dataTrans(train,'train')
#df_test = dataTrans(test,'')

In [ ]:
df_train[df_train.country_province == 'Zimbabwe'] #
#df_train.tail()

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities,country+province,Lat,Long,day_from_jan_first,...,min,max,stp,slp,dewp,rh,ah,wdsp,prcp,fog
0,1,NaN,Afghanistan,2020-01-22,0.0,0.0,Afghanistan-,33.0,65.0,22,...,33.6,54.9,999.9,1024.3,27.4,0.545709,0.186448,9.4,0.00,0
1,2,NaN,Afghanistan,2020-01-23,0.0,0.0,Afghanistan-,33.0,65.0,23,...,32.7,55.9,999.9,1020.8,22.8,0.461259,0.163225,14.9,99.99,1
2,3,NaN,Afghanistan,2020-01-24,0.0,0.0,Afghanistan-,33.0,65.0,24,...,36.9,43.2,999.9,1018.6,34.5,0.801794,0.325375,10.4,0.17,1
3,4,NaN,Afghanistan,2020-01-25,0.0,0.0,Afghanistan-,33.0,65.0,25,...,37.9,56.3,999.9,1018.0,37.8,0.728175,0.214562,6.1,0.57,1
4,5,NaN,Afghanistan,2020-01-26,0.0,0.0,Afghanistan-,33.0,65.0,26,...,36.1,53.1,999.9,1014.8,33.2,0.685513,0.231656,10.8,0.00,1


# Time Series Regression
## Rolling window regression
1. Select one country to work with ('Afghanistan')
2. Create rolling windows=10 based on 70 days of observation
3. Perform regression on rolling windows
4. Select 10 country_regions to work with (first 10)
5. Repeat step 2 to 3

In [ ]:
#train_cum.country_province.unique()

# 1. Select one country to work with ('Germany')
df = train_filtered[train_filtered.country_province == 'Afghanistan']#train_cum[train_cum.country_province == 'Germany']
df.shape

df['ConfirmedCases'].plot.line(logy=True)



In [ ]:

# 2. Create rolling windows=10 based on 70 days of observation
rolling_mean = df[['Date','cumCases']].rolling(10).mean()
df[['ConfirmedCases','Fatalities','cumCases','cumDeath']].plot.line(logy=True)
rolling_mean.plot.line(logy=True)


# 3. Perform random forest regression on rolling windows

# set up dataset for train, test, and validation
X = rolling_mean.fillna(0)
y = df.cumCases

X_train = X[X.index < 50]
y_train = y[y.index < 50]

X_test = X[X.index >= 50]
y_test = y[y.index >= 50]

from sklearn.ensemble.forest import RandomForestRegressor


# build our RF model
RF_Model = RandomForestRegressor(n_estimators=100,
                                 max_features=1,
                                 oob_score=True)

# let's get the labels and features in order to run our model fitting
labels = y_train#[:, None]
features = X_train#[:, None]

# Fit the RF model with features and labels.
rgr=RF_Model.fit(features, labels)

# Now that we've run our models and fit it, let's create
# dataframes to look at the results
X_test_predict=pd.DataFrame(
    rgr.predict(X_test)).rename(
    columns={0:'predicted_cases'}).set_index('predicted_cases')
X_train_predict=pd.DataFrame(
    rgr.predict(X_train)).rename(
    columns={0:'predicted_cases'}).set_index('predicted_cases')

# combine the training and testing dataframes to visualize
# and compare.
RF_predict = X_train_predict.append(X_test_predict)

In [ ]:
RF_predict.tail(50)#[['cumCases', 'predicted_cases']].plot()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV

# create models
estimators = [('Random Forest',RandomForestRegressor(random_state=42)),
         ('Lasso',LassoCV()),
         ('Gradient Boosting', HistGradientBoostingRegressor(random_state=0))]
stackingRegressor = StackingRegressor(estimators=estimators, final_estimator=RidgeCV())

In [ ]:
import time
import numpy as np
from sklearn.model_selection import cross_validate, cross_val_predict
